In [1]:
import numpy as np
import pandas as pd
import datetime
import math

def psi(bench, comp, group):
    lowercut_list=[]
    uppercut_list=[]
    ben_pct_list=[]
    comp_pct_list=[]
    psi_cut=[]
    
    ben_len_all=len(bench)
    comp_len_all=len(comp)
    
    lowercut_list.append('NULL')
    uppercut_list.append('NULL') 
    
    ben_nan_count = sum(math.isnan(x) for x in bench)
    comp_nan_count = sum(math.isnan(x) for x in comp)
    
    ben_nan_pct = (ben_nan_count+0.0001)/ben_len_all
    comp_nan_pct = (comp_nan_count+0.0001)/comp_len_all
    
    ben_pct_list.append(ben_nan_pct)
    comp_pct_list.append(comp_nan_pct) 
    psi_cut.append(round((ben_nan_pct-comp_nan_pct)*math.log(ben_nan_pct/comp_nan_pct),2))
    
    bench = [i for i in bench if str(i) != 'nan']
    comp = [i for i in comp if str(i) != 'nan']
    
    bench.sort()
    comp.sort()
    
    ben_len=len(bench)
    comp_len=len(comp)


    if len(set(bench))<=5:
        
        for i in range(0, len(set(bench))):           
            if i==0:
                 lowercut=sorted(set(bench))[i]-10000000
                 uppercut=sorted(set(bench))[i]
            else:
                 lowercut=sorted(set(bench))[i-1]                 
                 uppercut=sorted(set(bench))[i]
            lowercut_list.append(lowercut)
            uppercut_list.append(uppercut)
            ben_cnt = len([i for i in bench if i > lowercut and i<=uppercut])
            comp_cnt = len([i for i in comp if i > lowercut and i<=uppercut])
            ben_pct=(ben_cnt+0.0001)/ben_len_all
            comp_pct=(comp_cnt+0.0001)/comp_len_all
            ben_pct_list.append(ben_pct)
            comp_pct_list.append(comp_pct)
            psi_cut.append(round((ben_pct-comp_pct)*math.log(ben_pct/comp_pct),4))
            psi=sum(psi_cut)
    else:
        n=int(math.floor(ben_len/group))
        for i in range(1,group+1):
            if i==1:
                lowercut=bench[(i-1)*n]-10000000
            else:
                lowercut=bench[(i-1)*n]
            if i!=group:
                uppercut=bench[(i*n)]
            else:
                uppercut=bench[-1]+10000000
            lowercut_list.append(lowercut)
            uppercut_list.append(uppercut)
            ben_cnt = len([i for i in bench if i > lowercut and i<=uppercut])
            comp_cnt = len([i for i in comp if i > lowercut and i<=uppercut])
            
            ben_pct=(ben_cnt+0.0001)/ben_len_all
            comp_pct=(comp_cnt+0.0001)/comp_len_all
            ben_pct_list.append(ben_pct)
            comp_pct_list.append(comp_pct)
            psi_cut.append(round((ben_pct-comp_pct)*math.log(ben_pct/comp_pct),4))
    

    psi=sum(psi_cut)
    
    ben_pct_list = [round(i, 4) for i in ben_pct_list]
    comp_pct_list = [round(i, 4) for i in comp_pct_list]
    summary_table=pd.DataFrame({'Lowercut':lowercut_list,'Uppercut':uppercut_list,'ben_pct':ben_pct_list,
                                'com_pct':comp_pct_list,'IV':psi_cut})
    print(summary_table)
    print('Information Value:{:4f}'.format(psi))
    return summary_table, psi

In [2]:
Q2 = pd.read_excel(r'\\neptune\RAD\1 Projects\2020\Jul_2020\ssn 000-000-0000\B1_SC_EXPORT_V6_07-15-20_7764_JM.xlsx')

In [5]:
imputed_dic = { 
                'addrchangecount60month': (-1),
                'addrcurrenttimeoldest': (-1),
                'addrinputtimeoldest': (-1),
                'addronfilecount': (-1),
                'addrprevioustimeoldest': (-1),
                'assetindex': (-1),
                'confirmationinputdob': (-1),
                'derogcount12month': (-1),
                'derogseverityindex': (-1),
                'derogtimenewest': (-1),
                'evictiontimenewest': (-1),
                'inquirytelcom12month': (-1),
                'sourcevoterregistration': (-1),
                'ssnproblems': (-1),

}

In [6]:
#impute the Q2
for i in imputed_dic.keys():    
    Q2[i.upper()] = Q2[i.upper()].replace(imputed_dic[i], np.nan) 
    


In [4]:
for i in ('auto_score', 'crossindustryscore', 'shorttermlendingscore', 'FTScore'):
    if i == 'FTScore':
        Q2[i.upper()] = Q2[i.upper()].mask(((Q2[i.upper()] < 500) | (Q2[i.upper()] > 900)) , np.nan)
    else: 
        Q2[i.upper()]  = Q2[i.upper()].mask(((Q2[i.upper()] <= 500) | (Q2[i.upper()] > 900)) , np.nan)

In [7]:
#000 vs not000
#difference
ssn_000 = Q2[(Q2['BUYER 1\'s SSN'] == '000-00-0000')&(Q2['FICO'].isnull())]
ssn_not000 = Q2[(Q2['BUYER 1\'s SSN'] != '000-00-0000')&(Q2['FICO'].isnull())]
IV_table = {}
IV_dic = {}
for i in Q2.iloc[:,27:105].drop(Q2.iloc[:,27:105].dtypes[Q2.iloc[:,27:105].dtypes == 'object'].index.tolist(), axis = 1).columns.tolist():
    bench = ssn_000[i].tolist()
    comp = ssn_not000[i].tolist()
    print(i)
    summary_table, IV = psi(bench, comp, group=5)
    IV_dic = {i:IV}
    IV_table.update(IV_dic)   
    IV_DF = pd.DataFrame.from_dict(IV_table, orient='index',columns=['IV']).sort_values(by='IV',ascending=False)
IV_DF.to_csv(r'\\neptune\RAD\1 Projects\2020\Jul_2020\ssn 000-000-0000\ssn000vsnot0004.csv')

ACH_SATISFACTORY_AMT_EVER
  Lowercut Uppercut  ben_pct  com_pct   IV
0     NULL     NULL      1.0     0.98  0.0
Information Value:0.000000
ADDRCHANGECOUNT60MONTH
  Lowercut Uppercut  ben_pct  com_pct      IV
0     NULL     NULL   0.5832   0.2632  0.2500
1   -1e+07        0   0.1184   0.2829  0.1433
2        0        1   0.2085   0.3134  0.0428
3        1        1   0.0000   0.0000  0.0000
4        1        2   0.0561   0.0934  0.0190
5        2    1e+07   0.0338   0.0471  0.0044
Information Value:0.459500
ADDRCURRENTTIMEOLDEST
  Lowercut     Uppercut  ben_pct  com_pct      IV
0     NULL         NULL   0.5872   0.2752  0.2400
1   -1e+07            4   0.0958   0.1191  0.0050
2        4            9   0.0727   0.0934  0.0052
3        9           20   0.0829   0.1243  0.0168
4       20           45   0.0790   0.1429  0.0379
5       45  1.00004e+07   0.0823   0.2451  0.1778
Information Value:0.482700
ADDRINPUTTIMEOLDEST
  Lowercut     Uppercut  ben_pct  com_pct      IV
0     NULL         N

FIP8120
  Lowercut Uppercut  ben_pct  com_pct   IV
0     NULL     NULL      1.0      1.0  0.0
Information Value:0.000000
FTSCORE
  Lowercut Uppercut  ben_pct  com_pct    IV
0     NULL     NULL      1.0   0.7686  0.06
Information Value:0.060000
FACTORTRUST.HIT
  Lowercut Uppercut  ben_pct  com_pct      IV
0     NULL     NULL   0.0021   0.0107  0.0100
1   -1e+07        0   0.9979   0.8562  0.0217
Information Value:0.031700
ILN6220
  Lowercut Uppercut  ben_pct  com_pct   IV
0     NULL     NULL      1.0      1.0  0.0
Information Value:0.000000
ILN6230
  Lowercut Uppercut  ben_pct  com_pct   IV
0     NULL     NULL      1.0      1.0  0.0
Information Value:0.000000
ILN7470
  Lowercut Uppercut  ben_pct  com_pct   IV
0     NULL     NULL      1.0      1.0  0.0
Information Value:0.000000
INQUIRYTELCOM12MONTH
  Lowercut Uppercut  ben_pct  com_pct      IV
0     NULL     NULL   0.5832   0.2632  0.2500
1   -1e+07        0   0.3996   0.6834  0.1523
2        0        1   0.0172   0.0534  0.0411
Informa